In [1]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader
import csv
import random
import numpy as np
import cv2
import mimetypes
import fnmatch
import seaborn as sns
from collections import Counter, defaultdict
import torchvision
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import time
from progress.bar import IncrementalBar
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import confusion_matrix
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
import io
from pytorch_lightning.callbacks import Callback
from datetime import datetime, date, time
from PIL import Image
import itertools 
from sklearn.model_selection import train_test_split
import torchvision.models as models

from Sources.callbacks_2classes_x10 import plot_confusion_matrix
from torch.nn import functional as F
from Sources.callbacks_2classes_x10 import get_true_classes
from Sources.callbacks_2classes_x10 import get_predicted_classes
from Sources.callbacks_2classes_x10 import get_classes_probs
from Sources.callbacks_2classes_x10 import callback
from Sources.data_tools import ImageDataset_2cl

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

labels_map_2cl = {
    "Benign": 0,
    "Malignant": 1,
}

transform = transforms.Compose([
    transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageDataset_2cl('../Data/burnasyan_Br.csv', 'train_paths_x10.txt', transform = transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = ImageDataset_2cl('../Data/burnasyan_Br.csv', 'val_paths_x10.txt', transform = transform)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

Using cuda device


In [3]:
net = models.resnet50(pretrained=False)
net.fc = nn.Linear(2048, 1)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

net.apply(init_weights)

net = net.to(device)

/tmp/ipykernel_22918/2898625335.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [4]:
logdir = "../Logits/ResNet50_2cl_x10_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(logdir)
res_callback = callback()

In [5]:
pos_weight = torch.from_numpy(np.array([0.5])).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(net.parameters(), lr=1e-5, betas=(0.9, 0.999))

In [6]:
epochs_num = 50

for epoch in tqdm_notebook(range(epochs_num), desc='epochs'):  # loop over the dataset multiple times
    
    res_callback.on_epoch_begin(epoch) 
    
    running_loss = 0.0
    
    classes = []
    true_classes= []

    for data in tqdm_notebook(train_dataloader, desc='one epoch training'):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        true_classes.append(labels.cpu().detach().numpy().astype(int))
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        if outputs.shape > torch.Size([1]):
            outputs = outputs.squeeze()
        if outputs.shape < torch.Size([1]):
                outputs = outputs.unsqueeze(0)
        
        # СИГМОИДА И ГРАНИЦА
        probs = torch.sigmoid(outputs)
        probs = probs.cpu().detach().numpy().astype(float)
        classes.append(probs) 
        
        outputs = outputs.to(device)
#         print(outputs.shape, labels.shape)
#         print(classes)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    running_loss /= len(true_classes)
    
    val_classes = []
    val_true_classes = []
    val_loss = 0.0
    
    for data in tqdm_notebook(val_dataloader, desc='validation'):
        # get the inputs; data is a list of [inputs, labels]
        val_inputs, val_labels = data
        val_true_classes.append(val_labels.cpu().detach().numpy().astype(int))
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
    
        # forward 
        with torch.no_grad():
            val_outputs = net(val_inputs)
            if val_outputs.shape > torch.Size([1]):
                val_outputs = val_outputs.squeeze()
            if val_outputs.shape < torch.Size([1]):
                val_outputs = val_outputs.unsqueeze(0)
            loss = criterion(val_outputs, val_labels.float())
            val_loss += loss.item()
        
        val_probs = torch.sigmoid(val_outputs)
        val_probs = val_probs.cpu().detach().numpy().astype(float)
        val_classes.append(val_probs) 
        
    val_loss /= len(val_true_classes)

    res_callback.on_epoch_end(true_classes, classes, val_true_classes, val_classes,
                          ["Benign", "Malignant"],
                          running_loss, val_loss, writer)
    
    if epoch % 10 == 0:
        torch.save(net.state_dict(), "../Logits/SavedNN/Saved_ResNet50_2cl_x10/" + str(epoch))

print('Finished Training')

epochs:   0%|          | 0/50 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1210 [00:00<?, ?it/s]

validation:   0%|          | 0/505 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1210 [00:00<?, ?it/s]

validation:   0%|          | 0/505 [00:00<?, ?it/s]

...

validation:   0%|          | 0/505 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1210 [00:00<?, ?it/s]

validation:   0%|          | 0/505 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1210 [00:00<?, ?it/s]

KeyboardInterrupt: 

### ResNet50 is overfitting on val until it gets to plato on train. Lets try ResNet34. 
### I also changed the parts of train-val-test split

In [11]:
net = models.resnet34(pretrained=False)
net.fc = nn.Linear(512, 1)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

net.apply(init_weights)

net = net.to(device)

/tmp/ipykernel_22918/4014021685.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [12]:
logdir = "../Logits/ResNet34_2cl_x10_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(logdir)
res_callback = callback()

In [13]:
pos_weight = torch.from_numpy(np.array([0.5])).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(net.parameters(), lr=1e-5, betas=(0.9, 0.999))

In [ ]:
epochs_num = 50

for epoch in tqdm_notebook(range(epochs_num), desc='epochs'):  # loop over the dataset multiple times
    
    res_callback.on_epoch_begin(epoch) 
    
    running_loss = 0.0
    
    classes = []
    true_classes= []

    for data in tqdm_notebook(train_dataloader, desc='one epoch training'):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        true_classes.append(labels.cpu().detach().numpy().astype(int))
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        if outputs.shape > torch.Size([1]):
            outputs = outputs.squeeze()
        if outputs.shape < torch.Size([1]):
                outputs = outputs.unsqueeze(0)
        
        # СИГМОИДА И ГРАНИЦА
        probs = torch.sigmoid(outputs)
        probs = probs.cpu().detach().numpy().astype(float)
        classes.append(probs) 
        
        outputs = outputs.to(device)
#         print(outputs.shape, labels.shape)
#         print(classes)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    running_loss /= len(true_classes)
    
    val_classes = []
    val_true_classes = []
    val_loss = 0.0
    
    for data in tqdm_notebook(val_dataloader, desc='validation'):
        # get the inputs; data is a list of [inputs, labels]
        val_inputs, val_labels = data
        val_true_classes.append(val_labels.cpu().detach().numpy().astype(int))
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
    
        # forward 
        with torch.no_grad():
            val_outputs = net(val_inputs)
            if val_outputs.shape > torch.Size([1]):
                val_outputs = val_outputs.squeeze()
            if val_outputs.shape < torch.Size([1]):
                val_outputs = val_outputs.unsqueeze(0)
            loss = criterion(val_outputs, val_labels.float())
            val_loss += loss.item()
        
        val_probs = torch.sigmoid(val_outputs)
        val_probs = val_probs.cpu().detach().numpy().astype(float)
        val_classes.append(val_probs) 
        
    val_loss /= len(val_true_classes)

    res_callback.on_epoch_end(true_classes, classes, val_true_classes, val_classes,
                          ["Benign", "Malignant"],
                          running_loss, val_loss, writer)
    
    if epoch % 10 == 0:
        torch.save(net.state_dict(), "../Logits/SavedNN/Saved_ResNet34_2cl_x10/" + str(epoch))

print('Finished Training')

epochs:   0%|          | 0/50 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

...

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

### Results: likewise
### Lets try ResNet18

In [3]:
net = models.resnet18(pretrained=False)
net.fc = nn.Linear(512, 1)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

net.apply(init_weights)

net = net.to(device)

/tmp/ipykernel_26232/1825597982.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [4]:
logdir = "../Logits/ResNet18_2cl_x10_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(logdir)
res_callback = callback()

In [5]:
pos_weight = torch.from_numpy(np.array([0.5])).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(net.parameters(), lr=1e-4, betas=(0.9, 0.999))

In [6]:
epochs_num = 10

for epoch in tqdm_notebook(range(epochs_num), desc='epochs'):  # loop over the dataset multiple times
    
    res_callback.on_epoch_begin(epoch) 
    
    running_loss = 0.0
    
    classes = []
    true_classes= []
    
    if epoch == 5:
        optimizer = optim.Adam(net.parameters(), lr=1e-5, betas=(0.9, 0.999))

    for data in tqdm_notebook(train_dataloader, desc='one epoch training'):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        true_classes.append(labels.cpu().detach().numpy().astype(int))
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        if outputs.shape > torch.Size([1]):
            outputs = outputs.squeeze()
        if outputs.shape < torch.Size([1]):
                outputs = outputs.unsqueeze(0)
        
        # СИГМОИДА И ГРАНИЦА
        probs = torch.sigmoid(outputs)
        probs = probs.cpu().detach().numpy().astype(float)
        classes.append(probs) 
        
        outputs = outputs.to(device)
#         print(outputs.shape, labels.shape)
#         print(classes)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    running_loss /= len(true_classes)
    
    val_classes = []
    val_true_classes = []
    val_loss = 0.0
    
    for data in tqdm_notebook(val_dataloader, desc='validation'):
        # get the inputs; data is a list of [inputs, labels]
        val_inputs, val_labels = data
        val_true_classes.append(val_labels.cpu().detach().numpy().astype(int))
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
    
        # forward 
        with torch.no_grad():
            val_outputs = net(val_inputs)
            if val_outputs.shape > torch.Size([1]):
                val_outputs = val_outputs.squeeze()
            if val_outputs.shape < torch.Size([1]):
                val_outputs = val_outputs.unsqueeze(0)
            loss = criterion(val_outputs, val_labels.float())
            val_loss += loss.item()
        
        val_probs = torch.sigmoid(val_outputs)
        val_probs = val_probs.cpu().detach().numpy().astype(float)
        val_classes.append(val_probs) 
        
    val_loss /= len(val_true_classes)

    res_callback.on_epoch_end(true_classes, classes, val_true_classes, val_classes,
                          ["Benign", "Malignant"],
                          running_loss, val_loss, writer)
    
    torch.save(net.state_dict(), "../Logits/SavedNN/Saved_ResNet18_2cl_x10/" + str(epoch))

print('Finished Training')

epochs:   0%|          | 0/10 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

...

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1613 [00:00<?, ?it/s]

validation:   0%|          | 0/202 [00:00<?, ?it/s]

Finished Training


## Лучший результат - на 5 эпохе